# Time for Search and RAG

## Connect to Weaviate

In [ ]:
import weaviate

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local()

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [ ]:
import json
def print_properties(item):
    print(
        json.dumps(
            item.properties,
            indent=2, sort_keys=True, default=str
        )
    )

In [ ]:
# client.query.

## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [ ]:
# from weaviate.classes.query import Filter

# client.query.

## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [ ]:
# client.query.

## Retrieval Augmented Generation!!!
[Docs - RAG](https://weaviate.io/developers/weaviate/search/generative)

In [ ]:
# client.generate